In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit
%matplotlib inline

In [ ]:
df = pd.read_csv("large.csv")

In [ ]:
class TDQ_Large():
    """
    function approximation using temporal difference Q learning
    """
    
    def __init__(self, nactions, nstates, alpha, gamma,data,episode_length, ld):
        self.Q = np.zeros((nstates, nactions)) #initial Q
        self.z = np.zeros((nstates, nactions)) #initial eligibility trace
        self.nactions = nactions 
        self.nstates = nstates
        self.alpha = alpha # learning rate
        self.gamma = gamma # discount
        self.ld = ld # lambda
        self.data = data # input data containing all observed states
        self.episode_length = episode_length #length of each episode in the data

  
    def value(self, state, action):
        """returns the vale for a given state and action"""
        return self.Q[state, action]

   
    def delta(self, pstate, paction, reward, state, action):
        
        """
        Delta function used to calculate the main error correlation bewteen the previous Q state and 
        the current Q state
        
        """
        return reward + (self.gamma * self.value(state,action)) - self.value(pstate,paction)

    
    def train(self, pstate, paction, reward, state, action, delta = None):
        
        """
        A single step of reinforcement learning.
        """

        if delta is None:
            delta = self.delta(pstate, paction, reward, state, action)

        self.z[pstate,paction] += 1.0 # replacing traces
        
        # update Q and z
        self.Q += self.alpha * delta * self.z
        self.z *= (self.gamma * self.ld)

   
    def learn(self, verbose = True):
        
        """
        main function for iterating over the episodes in the data
        """
        
        # length of episode in the data
        episode_length = self.episode_length
        
        # for-loop for each episode
        for i in range(int(len(self.data)/episode_length)):
            
            t = []
            episode_done = False
            episode = 0
            
            # while loop to iterate over all data in the episode
            while not episode_done:
                

                s = self.data[i*episode_length+episode,0]
                a = self.data[i*episode_length+episode,1]-1
                r = self.data[i*episode_length+episode,2]
                sp = self.data[i*episode_length+episode,3]
                na = self.data[i*episode_length+episode+1,1]-1
                t.append([s,a,r,sp,na])
                             
                episode+= 1
                if episode == episode_length-2:
                    episode_done = True
            
            # update our Q and z values during each step in each episode
            for (previous, paction, reward, state, action) in t:
                    self.train(previous, paction, reward, state, action)
            if verbose:
                print (i)
            



In [ ]:
data = df.values.astype(int)

In [ ]:
td = TDQ2(8,1757600,0.2,0.99,data,100,0.99)
td.learn()

In [ ]:
policy = np.argmax(td.V,axis=1)+1

In [ ]:
np.savetxt("large.policy",policy)